In [1]:
#Tạo đường dẫn chung để đọc utils
import sys
import os

#Tạo đường dẫn để gọi đến utils dễ hơn
project_root = os.path.abspath(os.path.join("..", '..', 'src'))
sys.path.append(project_root)

print(f"Đã thêm đường dẫn: {project_root}")
print(f"Thư mục hiện tại: {os.getcwd()}")

# Utils
from utils import load_chunks_from_json
from utils import save_chunks_to_json
from utils import bm25_tokenize, text_to_sparse_vector_bm25

Đã thêm đường dẫn: c:\DACNTT2526\IT_Project_2526\src
Thư mục hiện tại: c:\DACNTT2526\IT_Project_2526\data\TuTuongHoChiMinh


In [2]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTTextLine, LTChar
from pypdf import PdfReader

from sentence_transformers import SentenceTransformer


from rank_bm25 import BM25Okapi

import unicodedata
import torch
import numpy as np
import re

# ***Chunk văn bản***

In [3]:
def get_text_size_TrietHoc(pdf_path, total_pages):
    '''
        Hàm được sử dụng để kiểm tra xem toàn bộ các size text có trong tài liệu, chuẩn bị cho việc chunking
        Các tham số:
            - pdf_path: đường dẫn đến file pdf của môn Lịch sử đảng
        
        Hàm sẽ trả về 1 set chứa toàn bộ các text_size của tài liệu
    '''
    #Duyệt qua từng trang trong file pdf, bắt đầu từ trang 2 (bỏ qua trang bìa)
    size_set = set()
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        
        #Duyệt qua từng phần tử trong trang
        for element in page_layout:
            if isinstance(element, LTTextContainer):

                # Duyet qua từng dòng chữ trong phần tử
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        
                        #Lấy ra từng kí tự bên trong dòng chữ
                        for obj in text_line:
                            if isinstance(obj, LTChar):

                                #Lấy ra thông tin của kí tự
                                font = obj.fontname
                                text = obj.get_text()
                                text_size = obj.size

                                size_set.add(text_size)
    
    return size_set

# Xây dựng các hàm cần thiết cho quá trình xử lý văn bản

In [4]:
def parse_line(line):
    """
    Tách định danh và tiêu đề từ các dòng đề mục.

    Ví dụ:
    - 'Phần II  Những nguyên lý...' → ('Phần II', 'Những nguyên lý...')
    - 'Chương V  Vật chất...'       → ('Chương V', 'Vật chất...')
    - 'II- Nguồn gốc...'            → ('II-', 'Nguồn gốc...')
    - '3. Kết cấu...'               → ('3.', 'Kết cấu...')
    - 'b) Theo chiều sâu...'        → ('b)', 'Theo chiều sâu...')

    Nếu không khớp, trả về ('None', original_line)
    """

    line = line.strip()

    pattern = re.compile(
        r"""^
        (
            (Phần|Chương)\s+[IVXLCDM\d]+            # Phần II, Chương V
            |
            [IVXLCDM]+[-\.]                         # II- hoặc II.
            |
            \d+[\.\)]                               # 3. hoặc 3)
            |
            [a-zA-Z][\.\)]                          # a) hoặc A.
        )
        \s+(.+)                                     # phần còn lại là tiêu đề
        """, re.VERBOSE | re.IGNORECASE
    )

    match = pattern.match(line)
    if match:
        number = match.group(1).strip().rstrip(".-)")  # loại . - )
        title = match.group(3).strip()
        return number, title
    else:
        return "", line

In [ ]:
def normalize_text(text):
    """
    Chuyển chuỗi thành lowercase, xóa dấu, và xóa toàn bộ khoảng trắng.
    """
    # Lowercase
    text = text.lower()

    # Bỏ dấu
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')

    # Xóa toàn bộ khoảng trắng (space, tab, newline)
    text = re.sub(r'\s+', '', text)

    return text

In [ ]:
#Hàm dùng để nhận thông tin từ raw chunks, chuyển đổi sang DENSE vector để chuẩn bị nạp vào Database
def TrietHoc_raw_to_dense(part, chapter, section, subsection, sub_subsection, content, model, max_chars = 2048):

    #Tách các đề mục sang số mục và tiêu đề
    part_number, part_title = parse_line(part)
    chapter_number, chapter_title = parse_line(chapter)
    section_number, section_title = parse_line(section)
    subsection_number, subsection_title = parse_line(subsection)
    sub_subsection_number, sub_subsection_title = parse_line(sub_subsection)

    chunk_type = 'EXERCISES' if ("Câu hỏi ôn tập" in section or "câu hỏi ôn tập" in section) else 'THEORY'

    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content)

    result = []
    for i, content_piece in enumerate(content_chunks):
        chunk = {
            "id": f"TrietHoc_{normalize_text(part_number) if part_number else 0}_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if section_number else 0}_{sub_subsection_number if sub_subsection_number else 0}_{i}",
            "values": model.encode(content).tolist(),
            "metadata": {
                "subject": "Triết học Mác_Lênin",
                "part": part_number,
                "part_title": part_title,
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "sub_subsection": sub_subsection_number,
                "sub_subsection_title": sub_subsection_title,
                "content": content_piece,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

In [ ]:
def bm25_tokenize(text):
    return text.lower().split()

def text_to_sparse_vector_bm25(text, bm25, vocabulary):
    tokens = bm25_tokenize(text)
    vector = np.zeros(len(vocabulary))
    for i, word in enumerate(vocabulary):
        idf = bm25.idf.get(word, 0)
        tf = tokens.count(word)
        vector[i] = idf * tf
    indices = vector.nonzero()[0].tolist()
    values = vector[indices].tolist()
    return {"indices": indices, "values": values}

In [ ]:
#Hàm dùng để nhận thông tin từ raw chunks, chuyển đổi sang SPARSE vector để chuẩn bị nạp vào Database
def TrietHoc_raw_to_sparse(part, chapter, section, subsection, sub_subsection, content, bm25 = None, vocabluary = None, max_chars = 2048):

    #Tách các đề mục sang số mục và tiêu đề
    part_number, part_title = parse_line(part)
    chapter_number, chapter_title = parse_line(chapter)
    section_number, section_title = parse_line(section)
    subsection_number, subsection_title = parse_line(subsection)
    sub_subsection_number, sub_subsection_title = parse_line(sub_subsection)

    chunk_type = 'EXERCISES' if ("Câu hỏi ôn tập" in section or "câu hỏi ôn tập" in section) else 'THEORY'

    from langchain.text_splitter import RecursiveCharacterTextSplitter

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=max_chars,
        chunk_overlap=300,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    content_chunks = splitter.split_text(content)

    result = []
    for i, content_piece in enumerate(content_chunks):
        chunk = {
            "id": f"TrietHoc_{normalize_text(part_number) if part_number else 0}_{normalize_text(chapter_number) if chapter_number else 0}_{section_number if section_number else 0}_{subsection_number if section_number else 0}_{sub_subsection_number if sub_subsection_number else 0}_{i}",
            "sparse_values": text_to_sparse_vector_bm25(content_piece.strip(), bm25, vocabluary),
            "metadata": {
                "subject": "Triết học Mác_Lênin",
                "part": part_number,
                "part_title": part_title,
                "chapter": chapter_number,
                "chapter_title": chapter_title,
                "section": section_number,
                "section_title": section_title,
                "subsection": subsection_number,
                "subsection_title": subsection_title,
                "sub_subsection": sub_subsection_number,
                "sub_subsection_title": sub_subsection_title,
                "content": content_piece,
                "tokens": len(content_piece),
                "type": chunk_type
            }
        }
        result.append(chunk)

    return result

# Xử lý tách văn bản thành các chunks

In [5]:
#Phân chia các tiêu đề
SUBSECTION_PATTERN = re.compile(r"^\s*\d+[\.\)]")    # 1. hoặc 2)
SUBSECTION_PATTERN_CHAP2 = re.compile(r"^\s*[IVX]+-", re.IGNORECASE)   # 1. hoặc 2)
SUB_SUBSECTION_PATTERN = re.compile(r"^\s*[a-zA-Z][\.\)]")  # a), b.

In [6]:
#Hàm đung để kiểm tra xem tiêu đề hiện tại có phải sub section không
def is_subsection(line_chars, chapter_title):
    line = ''.join([obj.get_text() for obj in line_chars]).strip()
    if "Chương II  " in chapter_title:
        return bool(SUBSECTION_PATTERN_CHAP2.match(line))
    return bool(SUBSECTION_PATTERN.match(line))

#Hàm đung để kiểm tra xem tiêu đề hiện tại có phải subsub section không
def is_sub_subsection(line_chars, chapter_title):
    line = ''.join([obj.get_text() for obj in line_chars]).strip()
    if "Chương II " in chapter_title:
        return bool(SUBSECTION_PATTERN.match(line))
    return bool(SUB_SUBSECTION_PATTERN.match(line))

#Hàm đung để kiểm tra xem hiện tại có phải câu hỏi ôn tập hay không
def is_questions(line_chars):
    line = ''.join([obj.get_text() for obj in line_chars]).strip()
    return ("Câu hỏi ôn tập" in line or "câu hỏi ôn tập" in line)

In [8]:
#Hàm dùng để trả về diction chứa các phần raw_chunks của môn Triết Học
def get_raw_chunks(pdf_path, total_pages):
    is_content = False

    chunk_list = []

    part_title = curr_content_line =  ""
    chapter_title = curr_content_line =  ""
    section_title = curr_content_line =  ""
    subsection_title = curr_content_line =  ""
    sub_subsection_title = curr_content_line =  ""
    curr_content_line = curr_content_line =  ""

    for page_layout in extract_pages(pdf_path, page_numbers=range(3, total_pages)):
        
        #Duyệt qua từng phần tử trong trang
        for element in page_layout:
            if isinstance(element, LTTextContainer):

                # Duyet qua từng dòng chữ trong phần tử
                for text_line in element:
                    if isinstance(text_line, LTTextLine):

                        line_chars = [obj for obj in text_line if isinstance(obj, LTChar)]
                        
                        for i, obj in enumerate(line_chars):
                            font = obj.fontname
                            text = obj.get_text()
                            text_size = obj.size

                            #Xử lý các phần lớn
                            if text_size >= 22 :
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    part_title = chapter_title = section_title = subsection_title = sub_subsection_title = curr_content_line =  ""
                                
                                if i == len(line_chars) - 1:
                                    part_title = part_title + text + " "
                                else:   
                                    part_title += text
                            
                            #Xử lý các chương lớn
                            elif text_size == 18:
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    chapter_title = section_title = subsection_title = sub_subsection_title = curr_content_line =  ""
                                
                                if i == len(line_chars) - 1:
                                    chapter_title = chapter_title + text + " "
                                else:   
                                    chapter_title += text

                            #Xử lý các mục I, II, III,...
                            #Trong chương 2 thì đây sẽ là các phần A, B, C lớn (Dobe)
                            elif 13.5 < text_size < 14:
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    section_title = subsection_title = sub_subsection_title = curr_content_line =  ""
                                
                                if i == len(line_chars) - 1:
                                    section_title = section_title + text + " "
                                else:   
                                    section_title += text

                            #Xử lý phần câu hỏi ôn tập ở mỗi chương (Được xem là 1 mục trong chương)
                            if is_questions(line_chars):
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    section_title = subsection_title = sub_subsection_title = curr_content_line =  ""
                                
                                if i == len(line_chars) - 1:
                                    section_title = section_title + text + " "
                                else:   
                                    section_title += text

                            #Xử lý các mục con như 1, 2, 3
                            #Trong chương 2 thì đây sẽ là các phần I, II, III la mã
                            elif 13 <= text_size <= 13.1 and is_subsection(line_chars, chapter_title):
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    subsection_title = sub_subsection_title = curr_content_line =  ""

                                if i == len(line_chars) - 1:
                                    subsection_title = subsection_title + text + " "
                                else:   
                                    subsection_title += text

                            #Xử lý các mục con như a, b, c,..
                            #Trong chương 2 thì đây sẽ là các phần 1, 2, 3
                            elif 13 <= text_size <= 13.1 and is_sub_subsection(line_chars, chapter_title):
                                if is_content:
                                    
                                    chunk_list.append({
                                        "part": part_title,
                                        "chapter": chapter_title,
                                        "section": section_title,
                                        "sub_section": subsection_title,
                                        "sub_subsection": sub_subsection_title,
                                        "content": curr_content_line
                                    })
                                    is_content = False
                                    sub_subsection_title = curr_content_line =  ""

                                if i == len(line_chars) - 1:
                                    sub_subsection_title = sub_subsection_title + text + " "
                                else:   
                                    sub_subsection_title += text

                            # Xử lý các trường hợp nội dung còn lại
                            elif 11.6 <= text_size < 12.4:
                                is_content = True
                                if i == len(line_chars) - 1:
                                    curr_content_line = curr_content_line + text + " "
                                else:   
                                    curr_content_line += text
    if is_content:
        
        chunk_list.append({
            "part": part_title,
            "chapter": chapter_title,
            "section": section_title,
            "sub_section": subsection_title,
            "sub_subsection": sub_subsection_title,
            "content": curr_content_line
        })
    
    return chunk_list

# ***Tạo ra Dense Vector và Sparse Vector từ Raw***

## Tạo Dense Vector

In [ ]:
def chunk_with_embedding(raw_chunks, embedding_model):
    """
    Nhận vào danh sách raw_chunks, sau đó gọi hàm split_chunk_langchain để xử lý nhỏ ra và thêm embedding nếu cần.
    """
    final_chunks = []
    for chunk in raw_chunks:
        split_chunks = TrietHoc_raw_to_dense(
            chunk['part'],
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['sub_subsection'],
            chunk['content'],
            max_chars=2048,
            model=embedding_model
        )
        final_chunks.extend(split_chunks)

    return final_chunks

## Tạo Sparse Vector

In [ ]:
def chunk_with_sparse(raw_chunks, bm25, vocabulary):
    """
    Nhận vào danh sách raw_chunks, sau đó gọi hàm split_chunk_langchain để xử lý nhỏ ra và thêm embedding nếu cần.
    """
    final_chunks = []
    for chunk in raw_chunks:
        split_chunks = TrietHoc_raw_to_sparse(
            chunk['part'],
            chunk['chapter'],
            chunk['section'],
            chunk['sub_section'],
            chunk['sub_subsection'],
            chunk['content'],
            bm25=bm25,
            vocabluary=vocabulary
        )
        final_chunks.extend(split_chunks)

    return final_chunks

# ***Tạo các file json cần thiết***

In [9]:
import json
import os

def save_chunks_to_json(chunks, output_path):
    """
    Lưu danh sách các chunk (list of dict) ra file JSON.
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)  # Tạo folder nếu chưa có

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(chunks, f, ensure_ascii=False, indent=2)

    print(f"✅ Đã lưu {len(chunks)} chunks vào: {output_path}")

In [10]:
# Đọc data từ file json
def load_chunks_from_json(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        chunks = json.load(f)
    return chunks

# ***Main***

In [12]:
pdf_path = "./Tu_tuong_Ho_Chi_Minh.pdf"
API_KEY = "pcsk_6ZiTFz_C8w85GDBJV5TyQwKRBCJ1RyfZ1SAyJmtw8GEMte7uJQjCupSH9s1vBZXZZKniLB"

# Lấy số trang trong file PDF
reader = PdfReader(pdf_path)
total_pages = len(reader.pages)
print("Số trang trong PDF:", total_pages)

# Lấy ra toàn bộ fontsize của file PDF
size_set = get_text_size_TrietHoc(pdf_path, total_pages)
print(f'size set: {size_set}')

# - Phân loại các nội dung (Format của môn Triết học sẽ theo thứ tự: Phần -> Chương -> I, II, III,... -> 1, 2,... -> a, b, c,....):
#     - Kích thước chữ >= 22: Các phần và tiêu đề của các phần đó
#     - Kích thước chữ == 18: Chương và các tiêu đề của Chương
#     - Kích thước chữ == 14 (> 13.5 and < 14): Các mục I, II, III (Có thể là ABC đối với chương II và 123 đói với chương IV)
#     - Kích thước chữ == 13 (13 <= text_size < 13.1): Các mục con như 1, 2, 3,... (Đối với chương 2 sẽ là I, II, III,...) và các mục nhỏ hơn như a, b, c....
#     - Kích thước chữ == 12 (11.6 <= text_size < 12,4): Nội dung của môn

# Đọc file PDF, chuyển qua RAW chunks
# TT_HCM_raw_chunks = get_raw_chunks(pdf_path, total_pages)

# Đưa RAW chunks vào chuyển đổi sang Dense Vector
# embedding_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding")
# embedding_model.max_seq_length = 2048
# TrietHoc_dense_vector = chunk_with_embedding(TrietHoc_raw_chunks, embedding_model)

# # Đưa RAW chunks vào chuyển đổi sang Sparse Vector
# corpus_texts = [chunk["content"] for chunk in TrietHoc_raw_chunks]
# tokenized_corpus = [bm25_tokenize(text) for text in corpus_texts]
# bm25 = BM25Okapi(tokenized_corpus)
# vocabulary = list(bm25.idf.keys())
# TrietHoc_sparse_vector = chunk_with_sparse(TrietHoc_raw_chunks, bm25, vocabulary)


# #Lưu các Vector vừa tạo được vào file Json

# #Lưu Dense Vector
# save_chunks_to_json(TrietHoc_dense_vector, r"./TrietHoc_Dense.json")

#Lưu Sparse Vector
# save_chunks_to_json(TrietHoc_sparse_vector, r"./TrietHoc_Sparse.json")                    

# # Đọc file Json 
# TrietHoc_dense_vector = load_chunks_from_json(r"./TrietHoc_Dense.json")
# TrietHoc_sparse_vector = load_chunks_from_json(r"./TrietHoc_Sparse.json")


# # Upsert Vector lên các Database
# from pinecone.grpc import PineconeGRPC as Pinecone
# pc = Pinecone(api_key=API_KEY)

# dense_index = pc.Index(host="https://rag-subjects-dense-y0c06t1.svc.aped-4627-b74a.pinecone.io")
# dense_index.upsert(
#   vectors = TrietHoc_dense_vector,
#   namespace="triet-hoc"
# )

# sparse_index = pc.Index(host="https://rag-subjects-sparse-y0c06t1.svc.aped-4627-b74a.pinecone.io")
# sparse_index.upsert(
#   vectors = TrietHoc_sparse_vector,
#   namespace="triet-hoc"
# )

Số trang trong PDF: 150
size set: {1.0, 5.799999999999997, 5.800000000000011, 7.500000000000028, 7.5, 8.0, 7.0, 10.0, 11.999999999999993, 9.999999999999986, 13.0, 14.0, 14.000000000000057, 16.0, 13.000000000000057, 10.000000000000014, 13.000000000000014, 12.999999999999972, 13.000000000000028, 7.000000000000014, 7.499999999999943, 7.500000000000057, 9.999999999999972, 12.999999999999943, 12.999999999999986}


In [13]:
sorted_size_set = sorted(size_set)
print(sorted_size_set)

[1.0, 5.799999999999997, 5.800000000000011, 7.0, 7.000000000000014, 7.499999999999943, 7.5, 7.500000000000028, 7.500000000000057, 8.0, 9.999999999999972, 9.999999999999986, 10.0, 10.000000000000014, 11.999999999999993, 12.999999999999943, 12.999999999999972, 12.999999999999986, 13.0, 13.000000000000014, 13.000000000000028, 13.000000000000057, 14.0, 14.000000000000057, 16.0]


In [14]:
from collections import defaultdict

def sample_texts_by_size(pdf_path, total_pages, max_samples=5):
    samples = defaultdict(list)
    for page_layout in extract_pages(pdf_path, page_numbers=range(2, total_pages)):
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                for text_line in element:
                    if isinstance(text_line, LTTextLine):
                        for obj in text_line:
                            if isinstance(obj, LTChar):
                                text_size = obj.size
                                text = obj.get_text()
                                if len(samples[text_size]) < max_samples:
                                    samples[text_size].append(text_line.get_text().strip())
    # Print samples
    for size in sorted(samples):
        print(f"Size: {size}")
        for example in samples[size]:
            print(f"  {example}")
        print("-" * 40)

# Usage:
sample_texts_by_size(pdf_path, total_pages)

Size: 1.0
  lOMoARcPSD|38213158
  lOMoARcPSD|38213158
  lOMoARcPSD|38213158
  lOMoARcPSD|38213158
  lOMoARcPSD|38213158
----------------------------------------
Size: 5.799999999999997
  1 Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ XI, Nxb Chính trị quốc gia, Hà
  3 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2004, t.37, tr.474.
  4Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ V, Nxb Sự thật, Hà Nội, t. 3, tr.61.
  4 Đảng Cộng sản Việt Nam: Văn kiện Đại hội đại biểu toàn quốc lần thứ IX, Nxb Chính trị quốc gia, Hà
  3  Xem  GS,TS  Mạch  Quang   Thắng,  PGS,TS  Bùi  Đình  Phong,  TS   Chu Đức  Tính  (Đồng  Chủ  biên):
----------------------------------------
Size: 5.800000000000011
  1 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2001, t.12, tr. 9.
  2 Đảng Cộng sản Việt Nam: Văn kiện Đảng Toàn tập, Nxb Chính trị quốc gia, Hà Nội, 2004, t.30, tr.275.
  1Đảng Cộng sản Việ